In [ ]:
import cv2
import numpy as np
import random
import core.utils as utils
import tensorflow as tf
from tqdm import tqdm
import shutil
from core.yolov3 import YOLOv3, decode, compute_loss
from PIL import Image
from core.config import cfg
from core.dataset import Dataset
import os
import time
assert(tf.__version__=='2.0.0-beta1')

In [ ]:
"""Training YoloV3 on dark data"""

"""change __C.YOLO.CLASSES= "./data/classes/coco.names" to "./data/classes/darkdata.names" in core/config.py"""


trainset = Dataset('train')
logdir = "./data/logdemo"
steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = cfg.TRAIN.WARMUP_EPOCHS * steps_per_epoch
total_steps = cfg.TRAIN.EPOCHS * steps_per_epoch

input_tensor = tf.keras.layers.Input([416, 416,3])
conv_tensors = YOLOv3(input_tensor)

output_tensors = []
for i, conv_tensor in enumerate(conv_tensors):
    pred_tensor = decode(conv_tensor, i)
    output_tensors.append(conv_tensor)
    output_tensors.append(pred_tensor)

model = tf.keras.Model(input_tensor, output_tensors)

'''load weights of subset of classes from coco trained yolov3 weights'''

num = [0,1,2]
weight_num = []
for i in num:
    tmp=[(85*i),(85*i+1),(85*i+2),(85*i+3),(85*i+4),(85*i+4+2),(85*i+4+9),(85*i+4+40),(85*i+4+6),(85*i+4+3),(85*i+4+16),(85*i+4+57),(85*i+4+42),(85*i+4+17),(85*i+4+4),(85*i+4+1),(85*i+4+61)]
    weight_num.append(tmp)
req_act = []
for each in weight_num:
    for ea in each:
        req_act.append(ea)
        
wf = open('./yolov3.weights', 'rb')
major, minor, revision, seen, _ = np.fromfile(wf, dtype=np.int32, count=5)
j = 0
layer_bias = []
layer_weights = []
for i in range(75):
    conv_layer_name = 'conv2d_%d' %i if i > 0 else 'conv2d'
    bn_layer_name = 'batch_normalization_%d' %j if j > 0 else 'batch_normalization'

    conv_layer = model.get_layer(conv_layer_name)
    filters = conv_layer.filters
    k_size = conv_layer.kernel_size[0]
    in_dim = conv_layer.input_shape[-1]
    if i not in [58, 66, 74]:
            # darknet weights: [beta, gamma, mean, variance]
        bn_weights = np.fromfile(wf, dtype=np.float32, count=4 * filters)
            # tf weights: [gamma, beta, mean, variance]
        bn_weights = bn_weights.reshape((4, filters))[[1, 0, 2, 3]]
        bn_layer = model.get_layer(bn_layer_name)
        j += 1
        
    else:
        conv_bias = np.fromfile(wf, dtype=np.float32, count=255)
        layer_bias.append(conv_bias)
    if i not in [58, 66, 74]:        
        conv_shape = (filters, in_dim, k_size, k_size)
        conv_weights = np.fromfile(wf, dtype=np.float32, count=np.product(conv_shape))
        conv_weights = conv_weights.reshape(conv_shape).transpose([2, 3, 1, 0])
        conv_layer.set_weights([conv_weights])
        bn_layer.set_weights(bn_weights)
    elif i==58:
        conv_shape = (255,1024,1,1)
        conv_weights = np.fromfile(wf, dtype=np.float32, count=np.product(conv_shape))
        # tf shape (height, width, in_dim, out_dim)
        conv_weights = conv_weights.reshape(conv_shape).transpose([2, 3, 1, 0])
        layer_weights.append(conv_weights)
        conv_layer.set_weights([conv_weights[:,:,:,req_act], conv_bias[req_act]])
    elif i==66:
        conv_shape = (255,512,1,1)
        conv_weights = np.fromfile(wf, dtype=np.float32, count=np.product(conv_shape))
        # tf shape (height, width, in_dim, out_dim)
        conv_weights = conv_weights.reshape(conv_shape).transpose([2, 3, 1, 0])
        layer_weights.append(conv_weights)
        conv_layer.set_weights([conv_weights[:,:,:,req_act], conv_bias[req_act]])
    else:
        conv_shape = (255,256,1,1)
        conv_weights = np.fromfile(wf, dtype=np.float32, count=np.product(conv_shape))
        # tf shape (height, width, in_dim, out_dim)
        conv_weights = conv_weights.reshape(conv_shape).transpose([2, 3, 1, 0])
        layer_weights.append(conv_weights)
        conv_layer.set_weights([conv_weights[:,:,:,req_act], conv_bias[req_act]])
#print(len(wf.read()))
assert len(wf.read()) == 0, 'failed to read all data'
wf.close()

''' freezing all layers except output layers to fine tune on dark dataset'''
for layer in model.layers:
    if layer.name in model.output_names:
        layer.trainable = True
    else:
        layer.trainable = False
        
'''train the model from here'''
model.summary()
optimizer = tf.keras.optimizers.Adam()
if os.path.exists(logdir): shutil.rmtree(logdir)
writer = tf.summary.create_file_writer(logdir) ##

def train_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = model(image_data, training=True)
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        for i in range(3):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        tf.print("=> STEP %4d   lr: %.6f   giou_loss: %4.2f   conf_loss: %4.2f   "
                 "prob_loss: %4.2f   total_loss: %4.2f" %(global_steps, optimizer.lr.numpy(),
                                                          giou_loss, conf_loss,
                                                          prob_loss, total_loss))
        # update learning rate
        global_steps.assign_add(1)
        if global_steps < warmup_steps:
            lr = global_steps / warmup_steps *cfg.TRAIN.LR_INIT
        else:
            lr = cfg.TRAIN.LR_END + 0.5 * (cfg.TRAIN.LR_INIT - cfg.TRAIN.LR_END) * (
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi))
            )
        optimizer.lr.assign(lr.numpy())

        # writing summary data to logs to display progress in tensor board
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
            tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
        writer.flush()

for epoch in range(cfg.TRAIN.EPOCHS):
    print("epoch %4d/%4d begins here:"%(epoch+1,cfg.TRAIN.EPOCHS)," steps per epoch: ",steps_per_epoch)
    for image_data, target,unhandled in trainset:
        #print(target)
        train_step(image_data, target)
    model.save_weights("./yolov3_dark")